In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, sys
from datetime import date
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

pd.set_option("display.max_rows", 100, "display.max_columns", 100)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

def p(s):
    print(s)
    
pd.DataFrame.len = pd.Index.len = lambda x: print(len(x))

In [2]:
# Partition training, dev, and test data
seasonHist = pd.read_csv('seasonHist.csv')
X = seasonHist \
    .loc[:,seasonHist.columns.str.contains('-\d+') | seasonHist.columns.str.match('fanPts')] \
    .drop('fanPts',axis=1) \
    .fillna(0)
y = seasonHist.fanPts

X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(X, y, test_size=0.3, random_state=0)
X_dev_raw, X_test_raw, y_dev_raw, y_test_raw = train_test_split(X_test_raw, y_test_raw, test_size=0.5, random_state=0)

# Manually downselect useful columns
keepCols = \
    X.columns.str.contains('fanPts') | \
    X.columns.str.contains('^complete_pass') | \
    X.columns.str.contains('rush_attempt') | \
    X.columns.str.contains('yards_gained') | \
    X.columns.str.contains('^touchdown')

X_train_select, X_test_select, y_train_select, y_test_select = train_test_split(X.loc[:,keepCols], y, test_size=0.3, random_state=0)
X_dev_select, X_test_select, y_dev_select, y_test_select = train_test_split(X_test_select, y_test_select, test_size=0.5, random_state=0)

NameError: name 'X_test' is not defined

### Linear Regression

#### Base model

In [ ]:
X_train, X_dev, y_train, y_dev = X_train_raw, X_dev_raw, y_train_raw, y_dev_raw

m = linear_model.LinearRegression()
m.fit(X_train,y_train)
y_pred = m.predict(X_dev)

plt.scatter(y_dev, y_pred)
plt.plot(y_dev,y_dev,color='black')
plt.show()

mae = mean_absolute_error(y_dev, y_pred).round(1)
rmse = mean_squared_error(y_dev, y_pred, squared=False).round(1)
r2 = r2_score(y_dev, y_pred).round(2)
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'R^2: {r2}')

coefs = pd.DataFrame([X_train.columns,m.coef_]).transpose()
coefs.reindex(coefs[1].abs().sort_values(ascending=False).index).set_index(0).transpose()

### Ridge Regression

#### Base model

In [ ]:
X_train, X_dev, y_train, y_dev = X_train_raw, X_dev_raw, y_train_raw, y_dev_raw

m = linear_model.Ridge()
m.fit(X_train,y_train)
y_pred = m.predict(X_dev)

plt.scatter(y_dev, y_pred)
plt.plot(y_dev,y_dev,color='black')
plt.show()

mae = mean_absolute_error(y_dev, y_pred).round(1)
rmse = mean_squared_error(y_dev, y_pred, squared=False).round(1)
r2 = r2_score(y_dev, y_pred).round(2)
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'R^2: {r2}')

coefs = pd.DataFrame([X_train.columns,m.coef_]).transpose()
coefs = coefs.reindex(coefs[1].abs().sort_values(ascending=False).index).set_index(0).transpose()
coefs

#### GridSearchCV

In [ ]:
X_train, X_dev, y_train, y_dev = X_train_raw, X_dev_raw, y_train_raw, y_dev_raw

grid = GridSearchCV(
    linear_model.Ridge(),
    {'alpha':[10**x for x in range(-4,6)],'normalize':[True,False]},
    n_jobs = -1,
    cv = 5
)

grid.fit(X_train,y_train)

print("Best parameters set found on development set:")
print(grid.best_params_)
     
# pd.DataFrame(grid.cv_results_)[['param_alpha','param_normalize','mean_test_score','std_test_score','rank_test_score']]

m = linear_model.Ridge(**grid.best_params_)
m.fit(X_train,y_train)
y_pred = m.predict(X_dev)

plt.scatter(y_dev, y_pred)
plt.plot(y_dev,y_dev,color='black')
plt.show()

mae = mean_absolute_error(y_dev, y_pred).round(1)
rmse = mean_squared_error(y_dev, y_pred, squared=False).round(1)
r2 = r2_score(y_dev, y_pred).round(2)
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'R^2: {r2}')

coefs = pd.DataFrame([X_train.columns,m.coef_]).transpose()
coefs = coefs.reindex(coefs[1].abs().sort_values(ascending=False).index).set_index(0).transpose()
coefs

### Lasso Regression

#### Base model

In [ ]:
X_train, X_dev, y_train, y_dev = X_train_raw, X_dev_raw, y_train_raw, y_dev_raw

m = linear_model.Lasso()
m.fit(X_train,y_train)
y_pred = regr.predict(X_dev)

plt.scatter(y_dev, y_pred)
plt.plot(y_dev,y_dev,color='black')
plt.show()

mae = mean_absolute_error(y_dev, y_pred).round(1)
rmse = mean_squared_error(y_dev, y_pred, squared=False).round(1)
r2 = r2_score(y_dev, y_pred).round(2)
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'R^2: {r2}')

coefs = pd.DataFrame([X_train.columns,regr.coef_]).transpose()
coefs = coefs.reindex(coefs[1].abs().sort_values(ascending=False).index).set_index(0).transpose()
coefs

#### GridSearchCV

In [ ]:
X_train, X_dev, y_train, y_dev = X_train_raw, X_dev_raw, y_train_raw, y_dev_raw

grid = GridSearchCV(
    linear_model.Lasso(),
    {'alpha':[10**x for x in range(-4,6)],'normalize':[True,False]},
    n_jobs = -1,
    cv = 5
)

grid.fit(X_train,y_train)

print("Best parameters set found on development set:")
print(grid.best_params_)
     
# pd.DataFrame(grid.cv_results_)[['param_alpha','param_normalize','mean_test_score','std_test_score','rank_test_score']]

m = linear_model.Lasso(**grid.best_params_)
m.fit(X_train,y_train)
y_pred = m.predict(X_dev)

plt.scatter(y_dev, y_pred)
plt.plot(y_dev,y_dev,color='black')
plt.show()

mae = mean_absolute_error(y_dev, y_pred).round(1)
rmse = mean_squared_error(y_dev, y_pred, squared=False).round(1)
r2 = r2_score(y_dev, y_pred).round(2)
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'R^2: {r2}')

coefs = pd.DataFrame([X_train.columns,m.coef_]).transpose()
coefs = coefs.reindex(coefs[1].abs().sort_values(ascending=False).index).set_index(0).transpose()
coefs

#### GridSearchCV, hand-selected features

In [ ]:
X_train, X_dev, y_train, y_dev = X_train_select, X_dev_select, y_train_select, y_dev_select

grid = GridSearchCV(
    linear_model.Lasso(),
    {'alpha':[10**x for x in range(-4,6)],'normalize':[True,False]},
    n_jobs = -1,
    verbose=7,
    cv = 5
)

grid.fit(X_train,y_train)

print("Best parameters set found on development set:")
print(grid.best_params_)
     
# pd.DataFrame(grid.cv_results_)[['param_alpha','param_normalize','mean_test_score','std_test_score','rank_test_score']]

m = linear_model.Lasso(**grid.best_params_)
m.fit(X_train,y_train)
y_pred = m.predict(X_dev)

plt.scatter(y_dev, y_pred)
plt.plot(y_dev,y_dev,color='black')
plt.show()

mae = mean_absolute_error(y_dev, y_pred).round(1)
rmse = mean_squared_error(y_dev, y_pred, squared=False).round(1)
r2 = r2_score(y_dev, y_pred).round(2)
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'R^2: {r2}')

coefs = pd.DataFrame([X_train.columns,m.coef_]).transpose()
coefs = coefs.reindex(coefs[1].abs().sort_values(ascending=False).index).set_index(0).transpose()
coefs

#### GridSearchCV, hand-selected features

In [ ]:
X_train, X_dev, y_train, y_dev = X_train_select, X_dev_select, y_train_select, y_dev_select

grid = GridSearchCV(
    linear_model.Lasso(),
    {'alpha':[10**x for x in range(-4,6)],'normalize':[True,False]},
    n_jobs = -1,
    verbose=7,
    cv = 5
)

grid.fit(X_train,y_train)

print("Best parameters set found on development set:")
print(grid.best_params_)
     
# pd.DataFrame(grid.cv_results_)[['param_alpha','param_normalize','mean_test_score','std_test_score','rank_test_score']]

m = linear_model.Lasso(**grid.best_params_)
m.fit(X_train,y_train)
y_pred = m.predict(X_dev)

plt.scatter(y_dev, y_pred)
plt.plot(y_dev,y_dev,color='black')
plt.show()

mae = mean_absolute_error(y_dev, y_pred).round(1)
rmse = mean_squared_error(y_dev, y_pred, squared=False).round(1)
r2 = r2_score(y_dev, y_pred).round(2)
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'R^2: {r2}')

coefs = pd.DataFrame([X_train.columns,m.coef_]).transpose()
coefs = coefs.reindex(coefs[1].abs().sort_values(ascending=False).index).set_index(0).transpose()
coefs

### Elastic Net Regression

#### Base model

In [ ]:
X_train, X_dev, y_train, y_dev = X_train_raw, X_dev_raw, y_train_raw, y_dev_raw

m = linear_model.ElasticNet()
m.fit(X_train,y_train)
y_pred = regr.predict(X_dev)

plt.scatter(y_dev, y_pred)
plt.plot(y_dev,y_dev,color='black')
plt.show()

mae = mean_absolute_error(y_dev, y_pred).round(1)
rmse = mean_squared_error(y_dev, y_pred, squared=False).round(1)
r2 = r2_score(y_dev, y_pred).round(2)
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'R^2: {r2}')

coefs = pd.DataFrame([X_train.columns,regr.coef_]).transpose()
coefs = coefs.reindex(coefs[1].abs().sort_values(ascending=False).index).set_index(0).transpose()
coefs

#### GridSearchCV

In [ ]:
X_train, X_dev, y_train, y_dev = X_train_raw, X_dev_raw, y_train_raw, y_dev_raw

grid = GridSearchCV(
    linear_model.ElasticNet(),
    {'alpha':[10**x for x in range(-4,6)],'normalize':[True,False]},
    n_jobs = -1,
    verbose=7,
    cv = 5
)

grid.fit(X_train,y_train)

print("Best parameters set found on development set:")
print(grid.best_params_)
     
# pd.DataFrame(grid.cv_results_)[['param_alpha','param_normalize','mean_test_score','std_test_score','rank_test_score']]

m = linear_model.ElasticNet(**grid.best_params_)
m.fit(X_train,y_train)
y_pred = m.predict(X_dev)

plt.scatter(y_dev, y_pred)
plt.plot(y_dev,y_dev,color='black')
plt.show()

mae = mean_absolute_error(y_dev, y_pred).round(1)
rmse = mean_squared_error(y_dev, y_pred, squared=False).round(1)
r2 = r2_score(y_dev, y_pred).round(2)
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'R^2: {r2}')

coefs = pd.DataFrame([X_train.columns,m.coef_]).transpose()
coefs = coefs.reindex(coefs[1].abs().sort_values(ascending=False).index).set_index(0).transpose()
coefs

#### Automatic CV

In [ ]:
X_train, X_dev, y_train, y_dev = X_train_raw, X_dev_raw, y_train_raw, y_dev_raw

m = linear_model.ElasticNetCV(l1_ratio=[.25,.5,.75], random_state=0)
m.fit(X_train,y_train)
print(f'Alpha: {m.alpha_}')
print(f'l1 ratio: {m.l1_ratio_}')

y_pred = m.predict(X_dev)

plt.scatter(y_dev, y_pred)
plt.plot(y_dev,y_dev,color='black')
plt.show()

mae = mean_absolute_error(y_dev, y_pred).round(1)
rmse = mean_squared_error(y_dev, y_pred, squared=False).round(1)
r2 = r2_score(y_dev, y_pred).round(2)
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'R^2: {r2}')

coefs = pd.DataFrame([X_train.columns,m.coef_]).transpose()
coefs = coefs.reindex(coefs[1].abs().sort_values(ascending=False).index).set_index(0).transpose()
coefs

#### GridSearchCV, hand-selected features

In [ ]:
X_train, X_dev, y_train, y_dev = X_train_select, X_dev_select, y_train_select, y_dev_select

grid = GridSearchCV(
    linear_model.ElasticNet(),
    {
        'alpha':[10**x for x in range(-4,6)],
        'normalize':[True,False],
        'l1_ratio':[.25,.5,.75]
    },
    n_jobs = -1,
    verbose=7,
    cv = 5
)

grid.fit(X_train,y_train)

print("Best parameters set found on development set:")
print(grid.best_params_)
     
# pd.DataFrame(grid.cv_results_)[['param_alpha','param_normalize','mean_test_score','std_test_score','rank_test_score']]

m = linear_model.ElasticNet(**grid.best_params_)
m.fit(X_train,y_train)
y_pred = m.predict(X_dev)

plt.scatter(y_dev, y_pred)
plt.plot(y_dev,y_dev,color='black')
plt.show()

mae = mean_absolute_error(y_dev, y_pred).round(1)
rmse = mean_squared_error(y_dev, y_pred, squared=False).round(1)
r2 = r2_score(y_dev, y_pred).round(2)
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'R^2: {r2}')

coefs = pd.DataFrame([X_train.columns,m.coef_]).transpose()
coefs = coefs.reindex(coefs[1].abs().sort_values(ascending=False).index).set_index(0).transpose()
coefs

#### Automatic CV, hand-selected features

In [ ]:
X_train, X_dev, y_train, y_dev = X_train_select, X_dev_select, y_train_select, y_dev_select

m = linear_model.ElasticNetCV(l1_ratio=[.25,.5,.75], random_state=0)
m.fit(X_train,y_train)
print(f'Alpha: {m.alpha_}')
print(f'l1 ratio: {m.l1_ratio_}')

y_pred = m.predict(X_dev)

plt.scatter(y_dev, y_pred)
plt.plot(y_dev,y_dev,color='black')
plt.show()

mae = mean_absolute_error(y_dev, y_pred).round(1)
rmse = mean_squared_error(y_dev, y_pred, squared=False).round(1)
r2 = r2_score(y_dev, y_pred).round(2)
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'R^2: {r2}')

coefs = pd.DataFrame([X_train.columns,m.coef_]).transpose()
coefs = coefs.reindex(coefs[1].abs().sort_values(ascending=False).index).set_index(0).transpose()
coefs

#### GridSearchCV, hand-selected features, force positive coefficients

In [ ]:
X_train, X_dev, y_train, y_dev = X_train_select, X_dev_select, y_train_select, y_dev_select

grid = GridSearchCV(
    linear_model.ElasticNet(),
    {
        'alpha':[10**x for x in range(-4,6)],
        'normalize':[True,False],
        'l1_ratio':[.25,.5,.75],
        'positive':[True]
    },
    n_jobs = -1,
    verbose=7,
    cv = 5
)

grid.fit(X_train,y_train)

print("Best parameters set found on development set:")
print(grid.best_params_)
     
# pd.DataFrame(grid.cv_results_)[['param_alpha','param_normalize','mean_test_score','std_test_score','rank_test_score']]

m = linear_model.ElasticNet(**grid.best_params_)
m.fit(X_train,y_train)
y_pred = m.predict(X_dev)

plt.scatter(y_dev, y_pred)
plt.plot(y_dev,y_dev,color='black')
plt.show()

mae = mean_absolute_error(y_dev, y_pred).round(1)
rmse = mean_squared_error(y_dev, y_pred, squared=False).round(1)
r2 = r2_score(y_dev, y_pred).round(2)
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'R^2: {r2}')

coefs = pd.DataFrame([X_train.columns,m.coef_]).transpose()
coefs = coefs.reindex(coefs[1].abs().sort_values(ascending=False).index).set_index(0).transpose()
coefs

#### Automatic CV, hand-selected features, force positive coefficients

In [ ]:
X_train, X_dev, y_train, y_dev = X_train_select, X_dev_select, y_train_select, y_dev_select

m = linear_model.ElasticNetCV(l1_ratio=[.25,.5,.75], positive=[True], random_state=0)
m.fit(X_train,y_train)
print(f'Alpha: {m.alpha_}')
print(f'l1 ratio: {m.l1_ratio_}')

y_pred = m.predict(X_dev)

plt.scatter(y_dev, y_pred)
plt.plot(y_dev,y_dev,color='black')
plt.show()

mae = mean_absolute_error(y_dev, y_pred).round(1)
rmse = mean_squared_error(y_dev, y_pred, squared=False).round(1)
r2 = r2_score(y_dev, y_pred).round(2)
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'R^2: {r2}')

coefs = pd.DataFrame([X_train.columns,m.coef_]).transpose()
coefs = coefs.reindex(coefs[1].abs().sort_values(ascending=False).index).set_index(0).transpose()
coefs

### Bayesian Ridge Regression

#### Base model

In [ ]:
X_train, X_dev, y_train, y_dev = X_train_raw, X_dev_raw, y_train_raw, y_dev_raw

m = linear_model.BayesianRidge()
m.fit(X_train,y_train)
y_pred = regr.predict(X_dev)

plt.scatter(y_dev, y_pred)
plt.plot(y_dev,y_dev,color='black')
plt.show()

mae = mean_absolute_error(y_dev, y_pred).round(1)
rmse = mean_squared_error(y_dev, y_pred, squared=False).round(1)
r2 = r2_score(y_dev, y_pred).round(2)
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'R^2: {r2}')

coefs = pd.DataFrame([X_train.columns,regr.coef_]).transpose()
coefs = coefs.reindex(coefs[1].abs().sort_values(ascending=False).index).set_index(0).transpose()
coefs

#### GridSearchCV

In [ ]:
X_train, X_dev, y_train, y_dev = X_train_raw, X_dev_raw, y_train_raw, y_dev_raw

grid = GridSearchCV(
    linear_model.BayesianRidge(),
    {
        'alpha_1':[10**x for x in range(-2,3)],
        'alpha_2':[10**x for x in range(-2,3)],
        'lambda_1':[10**x for x in range(-2,3)],
        'lambda_2':[10**x for x in range(-2,3)],
        'normalize':[True,False]
    },
    n_jobs = -1,
    verbose=7,
    cv = 3
)

grid.fit(X_train,y_train)

print("Best parameters set found on development set:")
print(grid.best_params_)
     
# pd.DataFrame(grid.cv_results_)[['param_alpha','param_normalize','mean_test_score','std_test_score','rank_test_score']]

m = linear_model.BayesianRidge(**grid.best_params_)
m.fit(X_train,y_train)
y_pred = m.predict(X_dev)

plt.scatter(y_dev, y_pred)
plt.plot(y_dev,y_dev,color='black')
plt.show()

mae = mean_absolute_error(y_dev, y_pred).round(1)
rmse = mean_squared_error(y_dev, y_pred, squared=False).round(1)
r2 = r2_score(y_dev, y_pred).round(2)
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'R^2: {r2}')

coefs = pd.DataFrame([X_train.columns,m.coef_]).transpose()
coefs = coefs.reindex(coefs[1].abs().sort_values(ascending=False).index).set_index(0).transpose()
coefs

#### GridSearchCV, hand-selected features

In [ ]:
X_train, X_dev, y_train, y_dev = X_train_select, X_dev_select, y_train_select, y_dev_select

grid = GridSearchCV(
    linear_model.BayesianRidge(),
    {
        'alpha_1':[10**x for x in range(-2,3)],
        'alpha_2':[10**x for x in range(-2,3)],
        'lambda_1':[10**x for x in range(-2,3)],
        'lambda_2':[10**x for x in range(-2,3)],
        'normalize':[True,False]
    },
    n_jobs = -1,
    verbose=7,
    cv = 3
)

grid.fit(X_train,y_train)

print("Best parameters set found on development set:")
print(grid.best_params_)
     
# pd.DataFrame(grid.cv_results_)[['param_alpha','param_normalize','mean_test_score','std_test_score','rank_test_score']]

m = linear_model.BayesianRidge(**grid.best_params_)
m.fit(X_train,y_train)
y_pred = m.predict(X_dev)

plt.scatter(y_dev, y_pred)
plt.plot(y_dev,y_dev,color='black')
plt.show()

mae = mean_absolute_error(y_dev, y_pred).round(1)
rmse = mean_squared_error(y_dev, y_pred, squared=False).round(1)
r2 = r2_score(y_dev, y_pred).round(2)
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'R^2: {r2}')

coefs = pd.DataFrame([X_train.columns,m.coef_]).transpose()
coefs = coefs.reindex(coefs[1].abs().sort_values(ascending=False).index).set_index(0).transpose()
coefs

#### GridSearchCV, hand-selected features, force positive coefficients

In [ ]:
X_train, X_dev, y_train, y_dev = X_train_select, X_dev_select, y_train_select, y_dev_select

grid = GridSearchCV(
    linear_model.BayesianRidge(),
    {
        'alpha_1':[10**x for x in range(-4,10)],
        'alpha_2':[10**x for x in range(-4,10)],
        'lambda_1':[10**x for x in range(-4,10)],
        'lambda_2':[10**x for x in range(-4,10)],
        'normalize':[True,False],
        'positive':[True]
    },
    n_jobs = -1,
    verbose=7,
    cv = 5
)

grid.fit(X_train,y_train)

print("Best parameters set found on development set:")
print(grid.best_params_)
     
# pd.DataFrame(grid.cv_results_)[['param_alpha','param_normalize','mean_test_score','std_test_score','rank_test_score']]

clf = linear_model.BayesianRidge(**grid.best_params_)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_dev)

plt.scatter(y_dev, y_pred)
plt.plot(y_dev,y_dev,color='black')
plt.show()

mae = mean_absolute_error(y_dev, y_pred).round(1)
rmse = mean_squared_error(y_dev, y_pred, squared=False).round(1)
r2 = r2_score(y_dev, y_pred).round(2)
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'R^2: {r2}')

coefs = pd.DataFrame([X_train.columns,clf.coef_]).transpose()
coefs = coefs.reindex(coefs[1].abs().sort_values(ascending=False).index).set_index(0).transpose()
coefs